In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

#The following is for test purposes
from sklearn import preprocessing
from sklearn import utils

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [2]:
df_bt = pd.read_csv('blocks_timestamps.csv') # Timestamps for each block
df_om = pd.read_csv('ordermatches_all.csv')  # Order details for each order (block)
df_pp = pd.read_csv('pepecash_prices.csv')   # USD value of pepecash over time (timeblock)
df_xp = pd.read_csv('xcp_prices.csv')        # USD value of xcp over time (timeblock)

In [3]:
#df_bt

In [4]:
#df_om

In [5]:
#df_pp

In [6]:
#df_xp

In [7]:
# Need to throw out all data that isn't xcp or pepecash value
# JK no we don't
# df_om.drop(df_om[df_om.ForwardAsset != ('PEPECASH' or 'XCP')].index, inplace=True)

In [8]:
#df_om_merge = pd.concat([df_om, df_bt], axis=1, join='inner')
df_om_merge = pd.merge(df_om, df_bt, on='Block', how='inner')

In [9]:
#df_om.insert
#df_om_merge = pd.merge(df_om, df_bt)
#df_om_merge = pd.concat([df_om, df_bt], on='Block', axis=1)
df_om_merge

,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp
0,428940,RAREPEPE,1.0,XCP,2.00,1473403689
1,429221,SHITCOINCARD,230.0,XCP,13.80,1473561677
2,429221,RAREPEPE,10.0,XCP,15.00,1473561677
3,429228,SHITCOINCARD,3.0,XCP,0.18,1473567368
4,429228,RAREPEPE,1.0,XCP,1.50,1473567368
...,...,...,...,...,...,...
25969,507166,PEPECASH,999.0,SISTNCHAPEPE,3.00,1517525368
25970,507184,DONTBEMENACE,1.0,PEPECASH,67.00,1517533848
25971,507187,BCCPEPE,1.0,PEPECASH,150.00,1517535355
25972,507187,SISTNCHAPEPE,1.0,PEPECASH,333.00,1517535355


In [10]:
df_om_merge.columns

Index(['Block', 'ForwardAsset', 'ForwardQuantity', 'BackwardAsset',
       'BackwardQuantity', 'Timestamp'],
      dtype='object')

In [11]:
'''#for row in df_om_merge:
df_om_merge2 = df_om_merge
for x, y in df_om_merge2.iterrows():
    check = False
    i = 0
    if(df_om_merge2.loc[x, 'BackwardAsset'] == 'XCP'):
        while check != True or i<519:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x] * df_xp.Price[i+1]
                df_om_merge2.BackwardAsset[x] = 'USD'
            i=i+1
        i=0
        check = False
    elif (df_om_merge2.loc[x, 'ForwardAsset'] == 'XCP'):
        while check != True or i<519:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.ForwardQuantity[x] = df_om_merge2.ForwardQuantity[x] * df_xp.Price[i+1]
                df_om_merge2.ForwardAsset[x] = 'USD'
            i=i+1
        i=0
        
    # PEPECASH Conversion
        check = False
    elif (df_om_merge2.loc[x, 'BackwardAsset'] == 'PEPECASH'):
        while check != True or i<490:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_pp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_pp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x] * df_pp.Price[i+1]
                df_om_merge2.BackwardAsset[x] = 'USD'
            i=i+1
        i=0
        check = False
    else:#(df_om_merge2.loc[x, 'ForwardAsset'] == 'PEPECASH'):
        while check != True or i<490:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_pp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_pp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.ForwardQuantity[x] = df_om_merge2.ForwardQuantity[x] * df_pp.Price[i+1]
                df_om_merge2.ForwardAsset[x] = 'USD'
            i=i+1
           
        
        #for label, row in df_xp.iterrows():
                #print(label)
                #print(row)
                #if(ts >= df_xp.Timestamp[x] and ts < df_xp.Timestamp[x+1]):
                    #print(x)
                    #df_om_merge.BackwardQuantity[i] = df_xp.Price[x] * df_om_merge.BackwardQuantity[i]
                    #df_om_merge.BackwardAsset = 'USD'
                
#print(df_om_merge.Timestamp)
'''
#Commented out because it takes forever

"#for row in df_om_merge:\ndf_om_merge2 = df_om_merge\nfor x, y in df_om_merge2.iterrows():\n    check = False\n    i = 0\n    if(df_om_merge2.loc[x, 'BackwardAsset'] == 'XCP'):\n        while check != True or i<519:\n            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):\n                check = True\n                df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x] * df_xp.Price[i+1]\n                df_om_merge2.BackwardAsset[x] = 'USD'\n            i=i+1\n        i=0\n        check = False\n    elif (df_om_merge2.loc[x, 'ForwardAsset'] == 'XCP'):\n        while check != True or i<519:\n            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):\n                check = True\n                df_om_merge2.ForwardQuantity[x] = df_om_merge2.ForwardQuantity[x] * df_xp.Price[i+1]\n               

In [12]:
# This dataset now has the USD equivalent of the XCP values and PEPECASH
#Load ordermatches_USD.csv because it is faster doing this
df_om_merge2 = pd.read_csv('ordermatches_USD.csv')   
df_om_merge2

,Unnamed: 0,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp
0,0,428940,RAREPEPE,1.000000,USD,6.660000,1473403689
1,1,429221,SHITCOINCARD,230.000000,USD,44.988000,1473561677
2,2,429221,RAREPEPE,10.000000,USD,48.900000,1473561677
3,3,429228,SHITCOINCARD,3.000000,USD,0.586800,1473567368
4,4,429228,RAREPEPE,1.000000,USD,4.890000,1473567368
...,...,...,...,...,...,...,...
25969,25969,507166,USD,41.135823,SISTNCHAPEPE,3.000000,1517525368
25970,25970,507184,DONTBEMENACE,1.000000,USD,2.758859,1517533848
25971,25971,507187,BCCPEPE,1.000000,USD,6.176550,1517535355
25972,25972,507187,SISTNCHAPEPE,1.000000,USD,13.711941,1517535355


In [13]:
'''df_om_merge2 = pd.read_csv('ordermatches_USD.csv')
for x, y in df_om_merge2.iterrows():
    if(df_om_merge2.loc[x, 'ForwardAsset'] == 'USD'):
        df_om_merge2.ForwardQuantity[x], df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x], df_om_merge2.ForwardQuantity[x]
'''
#Swap quantities - dollar on right, card on left
#Commented out because it takes forever

"df_om_merge2 = pd.read_csv('ordermatches_USD.csv')\nfor x, y in df_om_merge2.iterrows():\n    if(df_om_merge2.loc[x, 'ForwardAsset'] == 'USD'):\n        df_om_merge2.ForwardQuantity[x], df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x], df_om_merge2.ForwardQuantity[x]\n"

In [14]:
'''for x, y in df_om_merge2.iterrows():
    if(df_om_merge2.loc[x, 'ForwardAsset'] == 'USD'):
        df_om_merge2.ForwardAsset[x], df_om_merge2.BackwardAsset[x] = df_om_merge2.BackwardAsset[x], df_om_merge2.ForwardAsset[x]
'''
#Swap assets - same as above

"for x, y in df_om_merge2.iterrows():\n    if(df_om_merge2.loc[x, 'ForwardAsset'] == 'USD'):\n        df_om_merge2.ForwardAsset[x], df_om_merge2.BackwardAsset[x] = df_om_merge2.BackwardAsset[x], df_om_merge2.ForwardAsset[x]\n"

In [15]:
#df_om_merge2
#df_om_merge2.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\Ordered.csv')
df_om_merge3 = pd.read_csv('Ordered.csv')  
df_om_merge3.drop(df_om_merge2.columns[df_om_merge2.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df_om_merge3


,Unnamed: 0.1,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp
0,0,428940,RAREPEPE,1.000000,USD,6.660000,1473403689
1,1,429221,SHITCOINCARD,230.000000,USD,44.988000,1473561677
2,2,429221,RAREPEPE,10.000000,USD,48.900000,1473561677
3,3,429228,SHITCOINCARD,3.000000,USD,0.586800,1473567368
4,4,429228,RAREPEPE,1.000000,USD,4.890000,1473567368
...,...,...,...,...,...,...,...
25969,25969,507166,USD,41.135823,SISTNCHAPEPE,3.000000,1517525368
25970,25970,507184,DONTBEMENACE,1.000000,USD,2.758859,1517533848
25971,25971,507187,BCCPEPE,1.000000,USD,6.176550,1517535355
25972,25972,507187,SISTNCHAPEPE,1.000000,USD,13.711941,1517535355


In [16]:
#df_om_merge3['PricePerPepe'] = df_om_merge3['BackwardQuantity'] / df_om_merge3['ForwardQuantity']
#df_om_merge3.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\PPP.csv')
df_om_merge4 = pd.read_csv('PPP.csv') 
df_om_merge4
#Commented out because it takes a while, load from PPP to continue

,Unnamed: 0,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp,PricePerPepe
0,0,428940,RAREPEPE,1.0,USD,6.660000,1473403689,6.660000
1,1,429221,SHITCOINCARD,230.0,USD,44.988000,1473561677,0.195600
2,2,429221,RAREPEPE,10.0,USD,48.900000,1473561677,4.890000
3,3,429228,SHITCOINCARD,3.0,USD,0.586800,1473567368,0.195600
4,4,429228,RAREPEPE,1.0,USD,4.890000,1473567368,4.890000
...,...,...,...,...,...,...,...,...
25969,25969,507166,SISTNCHAPEPE,3.0,USD,41.135823,1517525368,13.711941
25970,25970,507184,DONTBEMENACE,1.0,USD,2.758859,1517533848,2.758859
25971,25971,507187,BCCPEPE,1.0,USD,6.176550,1517535355,6.176550
25972,25972,507187,SISTNCHAPEPE,1.0,USD,13.711941,1517535355,13.711941


In [17]:
#df_om_merge2.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\PPP.csv')
df_om_merge4 = pd.read_csv('PPP.csv')  
df_om_merge4.drop(df_om_merge4.columns[df_om_merge4.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df_om_merge4.rename(columns= {'ForwardAsset': 'CardName',
                               'ForwardQuantity': 'NumCards',
                               'BackwardAsset': 'Payment',
                               'BackwardQuantity': 'PriceUSD'}, inplace=True)
df_om_merge4 = df_om_merge4.reindex(columns=['Block', 'Timestamp', 'CardName', 'NumCards', 'Payment',
                                            'PriceUSD', 'PricePerPepe'])
df_om_merge4
#df_om_merge4.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\Cleanup.csv')

#change column names

,Block,Timestamp,CardName,NumCards,Payment,PriceUSD,PricePerPepe
0,428940,1473403689,RAREPEPE,1.0,USD,6.660000,6.660000
1,429221,1473561677,SHITCOINCARD,230.0,USD,44.988000,0.195600
2,429221,1473561677,RAREPEPE,10.0,USD,48.900000,4.890000
3,429228,1473567368,SHITCOINCARD,3.0,USD,0.586800,0.195600
4,429228,1473567368,RAREPEPE,1.0,USD,4.890000,4.890000
...,...,...,...,...,...,...,...
25969,507166,1517525368,SISTNCHAPEPE,3.0,USD,41.135823,13.711941
25970,507184,1517533848,DONTBEMENACE,1.0,USD,2.758859,2.758859
25971,507187,1517535355,BCCPEPE,1.0,USD,6.176550,6.176550
25972,507187,1517535355,SISTNCHAPEPE,1.0,USD,13.711941,13.711941


In [18]:
df_om_merge5 = df_om_merge4
df_om_merge5.sort_values(["CardName", "Timestamp"], axis=0, inplace=True)
#df_om_merge5.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\Sorted.csv')
df_om_merge4
#sort by cardname, then timestamp

,Block,Timestamp,CardName,NumCards,Payment,PriceUSD,PricePerPepe
1396,434445,1476546441,BACHELORPEPE,1.0,USD,17.040000,17.040000
1783,434852,1476797843,BACHELORPEPE,1.0,USD,12.450000,12.450000
3349,439210,1479306308,BACHELORPEPE,1.0,USD,8.840000,8.840000
4817,446214,1483329596,BACHELORPEPE,1.0,USD,38.400000,38.400000
10456,455933,1488760516,BACHELORPEPE,1.0,USD,29.000000,29.000000
...,...,...,...,...,...,...,...
7141,452381,1486713008,ZZZZPEPE,1.0,USD,5.256900,5.256900
8595,453684,1487461878,ZZZZPEPE,1.0,USD,10.465125,10.465125
18750,475017,1499647882,ZZZZPEPE,1.0,USD,12.955032,12.955032
18751,475025,1499650316,ZZZZPEPE,1.0,USD,12.955032,12.955032


In [19]:
len(df_om_merge4['CardName'].unique())
#check how many unique cards there are

1468

In [20]:
df_pn = pd.read_csv('pepe_number.csv')

In [21]:
df_pn

,CardName,NumIssued
0,RAREPEPE,300
1,GOXPEPE,1500
2,SHITCOINCARD,3000
3,PEPENATION,10000
4,DANKPEPE,420
...,...,...
1768,MCGROYPER,222
1769,KINGRINGPEPE,100
1770,HINAMATSURI,100
1771,SPACEXPEPE,200


In [22]:
# Adding the Number of pepe issed per pepe card name for df
df_om_merge5['NumberIssued'] = df_om_merge5['CardName'].map(df_pn.set_index('CardName')['NumIssued'])

In [23]:
df_om_merge5

,Block,Timestamp,CardName,NumCards,Payment,PriceUSD,PricePerPepe,NumberIssued
1396,434445,1476546441,BACHELORPEPE,1.0,USD,17.040000,17.040000,25.0
1783,434852,1476797843,BACHELORPEPE,1.0,USD,12.450000,12.450000,25.0
3349,439210,1479306308,BACHELORPEPE,1.0,USD,8.840000,8.840000,25.0
4817,446214,1483329596,BACHELORPEPE,1.0,USD,38.400000,38.400000,25.0
10456,455933,1488760516,BACHELORPEPE,1.0,USD,29.000000,29.000000,25.0
...,...,...,...,...,...,...,...,...
7141,452381,1486713008,ZZZZPEPE,1.0,USD,5.256900,5.256900,109.0
8595,453684,1487461878,ZZZZPEPE,1.0,USD,10.465125,10.465125,109.0
18750,475017,1499647882,ZZZZPEPE,1.0,USD,12.955032,12.955032,109.0
18751,475025,1499650316,ZZZZPEPE,1.0,USD,12.955032,12.955032,109.0


In [24]:
df_testing = df_om_merge5.drop(['CardName', 'Payment'], axis=1)
#df_testing['Timestamp'] = df_testing['Timestamp'] / 1000
df_testing['PriceUSD'] = round(df_testing['PriceUSD'], 3)
df_testing['PricePerPepe'] = round(df_testing['PricePerPepe'], 3)
df_testing_new = df_testing.dropna()
x = df_testing_new
y = df_testing_new["NumberIssued"]

In [25]:
x

,Block,Timestamp,NumCards,PriceUSD,PricePerPepe,NumberIssued
1396,434445,1476546441,1.0,17.040,17.040,25.0
1783,434852,1476797843,1.0,12.450,12.450,25.0
3349,439210,1479306308,1.0,8.840,8.840,25.0
4817,446214,1483329596,1.0,38.400,38.400,25.0
10456,455933,1488760516,1.0,29.000,29.000,25.0
...,...,...,...,...,...,...
7141,452381,1486713008,1.0,5.257,5.257,109.0
8595,453684,1487461878,1.0,10.465,10.465,109.0
18750,475017,1499647882,1.0,12.955,12.955,109.0
18751,475025,1499650316,1.0,12.955,12.955,109.0


In [26]:
y

1396      25.0
1783      25.0
3349      25.0
4817      25.0
10456     25.0
         ...  
7141     109.0
8595     109.0
18750    109.0
18751    109.0
19670    109.0
Name: NumberIssued, Length: 25703, dtype: float64

In [27]:
#These two lines convert output values to integers
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y)
#The next 3 lines are checks to ensure it has converted correctly

In [28]:
print(utils.multiclass.type_of_target(y))

continuous


In [29]:
print(utils.multiclass.type_of_target(y.astype('int')))

multiclass


In [30]:
print(utils.multiclass.type_of_target(encoded))

multiclass


In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.35, random_state=0)
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y_train)
lab_enc = preprocessing.LabelEncoder()
encoded1 = lab_enc.fit_transform(y_test)
tree = DecisionTreeClassifier(criterion='gini', max_depth=None, splitter='best', min_impurity_decrease=0.0005)
tree.fit(x_train, encoded)
output1 = tree.predict(x_test)
print(encoded1)
print(output1)
print("Accuracy on training set: {:.3f}".format(tree.score(x_train, encoded)))
print("Accuracy on test set: {:.3f}".format(tree.score(x_test, encoded1)))

[166 143  23 ...  79  22  22]
[175 151  24 ...  83  23  23]
Accuracy on training set: 0.989
Accuracy on test set: 0.001


In [32]:
accuracy = []
GoodScore = 0
BadScore = 0
TotalScore = 0

for i in output1:
    if(output1[i]/encoded1[i] >= 0.95 and output1[i]/encoded1[i] <= 1.05):
        accuracy.append(1)
        GoodScore=GoodScore+1
        TotalScore=TotalScore+1
        #print(output1[i]/encoded1[i])
    else:
        accuracy.append(0)
        BadScore=BadScore+1
        TotalScore=TotalScore+1
        #print(output1[i]/encoded1[i])

print("### TREES ###")       
print("Total Predicitons =", TotalScore)
print("Good Predicttions =", GoodScore)
print("Bad Predicttions =", BadScore)
actual_accuracy = GoodScore/TotalScore
print("Actual Accuracy within 5% = ", actual_accuracy)

### TREES ###
Total Predicitons = 8997
Good Predicttions = 6273
Bad Predicttions = 2724
Actual Accuracy within 5% =  0.6972324108036012


In [33]:
forest = RandomForestClassifier(n_estimators=8, random_state=2, criterion='entropy')
forest.fit(x_train,encoded)
output2 = forest.predict(x_test)
print(encoded1)
print(output2)
print("Accuracy on training set: {:.3f}".format(forest.score(x_train, encoded)))
print("Accuracy on test set: {:.3f}".format(forest.score(x_test, encoded1)))

[166 143  23 ...  79  22  22]
[175 151  23 ...  76  23  23]
Accuracy on training set: 0.995
Accuracy on test set: 0.012


In [34]:
accuracy = []
GoodScore = 0
BadScore = 0
TotalScore = 0

for i in output2:
    if(output2[i]/encoded1[i] >= 0.95 and output2[i]/encoded1[i] <= 1.05):
        accuracy.append(1)
        GoodScore=GoodScore+1
        TotalScore=TotalScore+1
        #print(output1[i]/encoded1[i])
    else:
        accuracy.append(0)
        BadScore=BadScore+1
        TotalScore=TotalScore+1
        #print(output1[i]/encoded1[i])
        
print("### RANDOM FOREST ###")       
print("Total Predicitons =", TotalScore)
print("Good Predicttions =", GoodScore)
print("Bad Predicttions =", BadScore)
actual_accuracy = GoodScore/TotalScore
print("Actual Accuracy within 5% = ", actual_accuracy)

### RANDOM FOREST ###
Total Predicitons = 8997
Good Predicttions = 6355
Bad Predicttions = 2642
Actual Accuracy within 5% =  0.7063465599644326


In [35]:
gnb = GaussianNB()
model = gnb.fit(x_train, encoded)
preds=gnb.predict(x_test)
print(encoded1)
print(preds)
print(accuracy_score(encoded1, preds))

[166 143  23 ...  79  22  22]
[23 23 23 ... 23 23 23]
0.008558408358341669


In [36]:
accuracy = []
GoodScore = 0
BadScore = 0
TotalScore = 0

for i in preds:
    if(preds[i]/encoded1[i] >= 0.95 and preds[i]/encoded1[i] <= 1.05):
        accuracy.append(1)
        GoodScore=GoodScore+1
        TotalScore=TotalScore+1
        #print(output1[i]/encoded1[i])
    else:
        accuracy.append(0)
        BadScore=BadScore+1
        TotalScore=TotalScore+1
        #print(output1[i]/encoded1[i])
print("### GAUSSIAN ###")       
print("Total Predicitons =", TotalScore)
print("Good Predicttions =", GoodScore)
print("Bad Predicttions =", BadScore)
actual_accuracy = GoodScore/TotalScore
print("Actual Accuracy within 5% = ", actual_accuracy)

### GAUSSIAN ###
Total Predicitons = 8997
Good Predicttions = 123
Bad Predicttions = 8874
Actual Accuracy within 5% =  0.013671223741247083


In [37]:
#output2.to_csv(r'C:\Programming\cmpe188\rarepepes\RandomForestPredictions.csv')
#np.savetxt(r'C:\Programming\cmpe188\rarepepes\RandomForestPredictions.csv', [output2], delimiter=',', fmt='%d', index=1)

df_Predictions = pd.read_csv('RandomForestPredictions.csv')

df_Predictions

,Predictions
0,175
1,151
2,23
3,23
4,23
...,...
8992,158
8993,104
8994,76
8995,23


In [38]:
#y_predictions = dtc.predict(x_test)
y_test

1080     1500.0
22645     777.0
18909     101.0
3699      100.0
25115     100.0
          ...  
24768     999.0
9397      300.0
20813     225.0
3058      100.0
24218     100.0
Name: NumberIssued, Length: 8997, dtype: float64

In [39]:
#print('Accuracy for given train:test split = ',accuracy_score(y_test, y_predictions))